<a href="https://colab.research.google.com/github/mcavol/AI-vs-Human-Text-Detector/blob/main/fine_tuned_ai_human_detector_44868.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is AI vs Human detector.
Firstly run only first block to downgrade NumPy library and then Restart Runtime!
After that you can run all other blocks. It takes dataset from this Kaggle page: https://www.kaggle.com/datasets/thedrcat/daigt-v2-train-dataset?select=train_v2_drcat_02.csv
But to have access to the file you need to have Kaggle account, so I just put it to my Google Drive for free access.
It use RoBERTa model from Hugging Face which is the best for this purpose around free models.
At the block 7 user can check his inputs.
But to run such training with this database you need to buy more powerfull Cloud computations from google, because free version has not enough RAM and CPU.



In [ ]:
#@title 1. Install Libraries and Downgrade NumPy
# We force numpy to a version before 2.0 to solve the compatibility issue.
!pip install transformers[torch] datasets evaluate pyarrow "numpy<2.0" -q

print("\n✅ Libraries installed and NumPy downgraded.")
print("‼️ IMPORTANT: You must now restart the runtime.")
print("Click 'Runtime' -> 'Restart runtime' from the menu above.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title 2. Load Your Uploaded CSV File
import pandas as pd
from datasets import Dataset

# Define the exact name of the file you uploaded.
file_path = 'https://drive.usercontent.google.com/download?id=16jly0gQIVd1j3k6nFe4RqiPsnJusDETz&export=download&authuser=0&confirm=t&uuid=d5cbe99a-7752-429a-acc6-523486206e5b&at=AN8xHoqbqa-s94wnMJSCfgD8qxGM:1750259149349'


print(f"Attempting to load your uploaded file: {file_path}")

try:
    # Use pandas to read the CSV file from your Colab session.
    df = pd.read_csv(file_path)

    # --- Data Processing ---
    # The column names 'text' and 'label' are already correct, which is perfect.
    # We will select just these two columns to ensure the dataset is clean.
    df_final = df[['text', 'label']]

    # Drop any rows that might have missing text, just in case.
    df_final.dropna(inplace=True)

    # Convert the pandas DataFrame into a Hugging Face Dataset.
    dataset = Dataset.from_pandas(df_final)

    # Shuffle the dataset to ensure a good mix for training.
    dataset = dataset.shuffle(seed=42)

    print("\n✅ File loaded and processed successfully!")
    print(f"Total samples loaded: {len(dataset)}")
    print(dataset)
    print("\nExample of Human-written text (label=0):")
    print(dataset.filter(lambda example: example['label'] == 0)[0]['text'])
    print("\nExample of AI-generated text (label=1):")
    print(dataset.filter(lambda example: example['label'] == 1)[0]['text'])

except FileNotFoundError:
    print(f"\n❌ ERROR: FileNotFoundError.")
    print(f"Please make sure the file named '{file_path}' is uploaded to your Colab session.")
    print("You can check the file list by clicking the folder icon on the left sidebar.")
    print("Also, please double-check for any typos in the filename.")
except Exception as e:
    print(f"\nAn error occurred while loading or processing the file: {e}")

Attempting to load your uploaded file: https://drive.usercontent.google.com/download?id=16jly0gQIVd1j3k6nFe4RqiPsnJusDETz&export=download&authuser=0&confirm=t&uuid=d5cbe99a-7752-429a-acc6-523486206e5b&at=AN8xHoqbqa-s94wnMJSCfgD8qxGM:1750259149349


<ipython-input-1-307226947>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.dropna(inplace=True)



✅ File loaded and processed successfully!
Total samples loaded: 44868
Dataset({
    features: ['text', 'label'],
    num_rows: 44868
})

Example of Human-written text (label=0):


Filter:   0%|          | 0/44868 [00:00<?, ? examples/s]

Thomas Wright once said, "To live is not simply to breathe; it is to know life and all of its components." One of those components is always education, which is a weapon in society. Without it, an individual could be justifiably categorized as defenseless. To take away a person's education would be dangerous and possibly detrimental to his or her success in life. American classrooms are designed to offer students a suitable education for general success, but they do not accommodate the obvious individuality among students. Due to the variety of students' lives, not every child is able to be present in the classroom every single day. Although in-person learning is more effective, because the medical issues and travel commitments of students are inevitable, students would benefit significantly from having the opportunity to attend classes from home.

Every person has known sickness and has not been fond of it, but the overall dislike of the common cold does not stop its occurrences. It i

Filter:   0%|          | 0/44868 [00:00<?, ? examples/s]

Dear State Senator,

I am writing to you today to express my support for abolishing the Electoral College and electing the President of the United States by popular vote. I believe that this is the only way to ensure that every vote counts and that the President is elected by the majority of the people.

The Electoral College is an outdated system that is no longer reflective of the American people. In the past, the Electoral College was seen as a way to protect the interests of small states. However, this is no longer the case. Today, the Electoral College gives an unfair advantage to a small number of swing states. This means that candidates can win the presidency without winning the popular vote.

This has happened twice in the past two decades. In 2000, George W. Bush won the presidency even though he lost the popular vote to Al Gore. In 2016, Donald Trump won the presidency even though he lost the popular vote to Hillary Clinton.

These results are undemocratic and unfair. They di

In [ ]:
#@title 3. Load RoBERTa Model and Tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#@title 4. Tokenize the Dataset
def tokenize_function(examples):
    # Truncate to 512 tokens, the max for roberta-base
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
# Remove the original text column as it's no longer needed after tokenization
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

# This dataset has about 33k examples. We'll use a larger subset for training.
train_size = 44000
eval_size = 868

small_train_dataset = tokenized_dataset.select(range(train_size))
small_eval_dataset = tokenized_dataset.select(range(train_size, train_size + eval_size))

print(f"Training dataset size: {len(small_train_dataset)}")
print(f"Evaluation dataset size: {len(small_eval_dataset)}")

Map:   0%|          | 0/44868 [00:00<?, ? examples/s]

Training dataset size: 2000
Evaluation dataset size: 373


In [ ]:
#@title 5. Configure and Run the Fine-Tuning
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

model_dir = "ai-human-detector-roberta-ardavey"

training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning complete!")

print("\nEvaluating the fine-tuned model...")
eval_results = trainer.evaluate()
print(f"Evaluation Accuracy: {eval_results['eval_accuracy']:.4f}")

Starting fine-tuning...


In [ ]:
#@title 6. Save Model and Create Inference Pipeline
from transformers import pipeline

final_model_path = "./final_ai_human_model_ardavey"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"Model saved to {final_model_path}")

# This pipeline will classify text as AI or HUMAN
detector = pipeline("text-classification", model=final_model_path, device=0)

print("\nInference pipeline created successfully!")


#@title 7. Test the Fine-Tuned Detector
ai_text = "The synergistic application of blockchain technology and artificial intelligence is poised to redefine digital trust paradigms."
human_text = "Wow, this pizza is incredible! I haven't had one this good in ages."

def pretty_print_result(text, result):
    # LABEL_0 is Human, LABEL_1 is AI
    label_map = {'LABEL_0': 'HUMAN', 'LABEL_1': 'AI'}
    label = label_map[result[0]['label']]
    score = result[0]['score']
    print(f"Text: '{text[:80]}...'")
    print(f"--> Verdict: {label} (Confidence: {score:.4f})\n")

print("\n--- Classifying Test Cases ---")
pretty_print_result(ai_text, detector(ai_text))
pretty_print_result(human_text, detector(human_text))

Model saved to ./final_ai_human_model_ardavey


Device set to use cuda:0



Inference pipeline created successfully!

--- Classifying Test Cases ---
Text: 'The synergistic application of blockchain technology and artificial intelligence...'
--> Verdict: AI (Confidence: 0.5873)

Text: 'Wow, this pizza is incredible! I haven't had one this good in ages....'
--> Verdict: AI (Confidence: 0.5543)



In [ ]:
#@title 7.  Check input from User
check = input("Write your text sample here:")
pretty_print_result(check, detector(check))

Write your text sample here:Yes, sure we could have a talk. I will be free tomorrow after 18:15 (Kyiv time, so after 17:15 in Paris), or on Tuesday any time after 16:00 (Kyiv time, so after 15:00 in Paris). Or tomorrow I also will be free between 16:00 and 17:15 (Kyiv time, so it's 15:00-16:15 in Paris).
Text: 'Yes, sure we could have a talk. I will be free tomorrow after 18:15 (Kyiv time, ...'
--> Verdict: AI (Confidence: 0.5718)

